#StackOverflow answer classifier

In [1]:
import re
import os
from html.parser import HTMLParser
from time import time
from xml.etree import ElementTree as etree
from xml.etree.ElementTree import Element

import numpy as np
import pandas as pd

In [2]:
class HTML2String(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.strict = False
        self.convert_charrefs= True
        self.text = []
    def handle_data(self, d):
        self.text.append(d)
    def get_data(self):
        return ''.join(self.text)

def html_to_string(html):
    s = HTML2String()
    s.feed(html)
    return s.get_data()

def count_code_lines(html):
    count_lines = 0
    codes = re.findall(r'<code>.*?</code>', html, re.DOTALL)
    for code in codes:
        text = html_to_string(code)
        lines = text.split('\n')
        lines = list(filter(lambda x: len(x) > 0, lines))
        count_lines += len(lines)
    return count_lines

def remove_code_from_html(html):
    nocode = html
    codes = re.findall(r'<code>.*?</code>', html, re.DOTALL)
    for code in codes:
        nocode = nocode.replace(code, '')
    return nocode

In [3]:
url_base = 'http://stackoverflow.com/'
url_base_q = url_base + 'questions/'
url_base_a = url_base + 'questions/'
url_base_u = url_base + 'users/'

In [4]:
qs_columns = ['id', 'author_id', 'date', 'title', 'text', 'score', 'view_count', 'answer_count', 'comment_count', 'code_line_count', 'url']
as_columns = ['id', 'author_id', 'question_id', 'date', 'text', 'score', 'accepted', 'comment_count', 'code_line_count', 'url']
us_columns = ['id', 'date', 'name', 'reputation', 'total_question_count', 'answered_question_count', 'total_answer_count', 'accepted_answer_count', 'url']

questions_df = pd.DataFrame(columns=qs_columns)
answers_df = pd.DataFrame(columns=as_columns)
users_df = pd.DataFrame(columns=us_columns)

In [5]:
def add_question(question):
    q_id = question.get('Id')
    author_id = question.get('OwnerUserId')
    date = question.get('CreationDate')
    title = question.get('Title')
    text = html_to_string(remove_code_from_html(question.get('Body')))
    score = int(question.get('Score'))
    view_count = int(question.get('ViewCount'))
    answer_count = int(question.get('AnswerCount'))
    comment_count = int(question.get('CommentCount'))
    code_line_count = count_code_lines(question.get('Body'))
    url = url_base_q + q_id
    
    info = [q_id, author_id, date, title, text, score, view_count, answer_count, comment_count, code_line_count, url]
    q_df = pd.DataFrame([info], columns=qs_columns)
    global questions_df
    questions_df = questions_df.append(q_df, ignore_index=True)

def add_answer(answer, accepted=False):
    a_id = answer.get('Id')
    author_id = answer.get('OwnerUserId')
    question_id = answer.get('ParentId')
    date = answer.get('CreationDate')
    text = html_to_string(remove_code_from_html(answer.get('Body')))
    score = int(answer.get('Score'))
    accepted = accepted
    comment_count = int(answer.get('CommentCount'))
    code_line_count = count_code_lines(answer.get('Body'))
    url = url_base_a + question_id + '/' + a_id

    info = [a_id, author_id, question_id, date, text, score, accepted, comment_count, code_line_count, url]
    a_df = pd.DataFrame([info], columns=as_columns)
    global answers_df
    answers_df = answers_df.append(a_df, ignore_index=True)

def add_user(user_id):
    u_id = user_id
    date = ''
    name = ''
    reputation = 0
    total_question_count = 0
    answered_question_count = 0
    total_answer_count = 0
    accepted_answer_count = 0
    url = url_base_u + u_id

    info = [u_id, date, name, reputation, total_question_count, answered_question_count, total_answer_count, accepted_answer_count, url]
    u_df = pd.DataFrame([info], columns=us_columns)
    global users_df
    users_df = users_df.append(u_df, ignore_index=True)

def update_user_info(user):
    user_id = user.get('Id')
    
    users_df.loc[users_df.id==user_id, 'date'] = user.get('CreationDate')
    users_df.loc[users_df.id==user_id, 'name'] = user.get('DisplayName')
    users_df.loc[users_df.id==user_id, 'reputation'] = int(user.get('Reputation'))

def update_user_question_counts(user_id, answered_question=False):
    users_df.loc[users_df.id==user_id, 'total_question_count'] += 1
    if answered_question:
        users_df.loc[users_df.id==user_id, 'answered_question_count'] += 1

def update_user_answer_counts(user_id, accepted_answer=False):
    users_df.loc[users_df.id==user_id, 'total_answer_count'] += 1
    if accepted_answer:
        users_df.loc[users_df.id==user_id, 'accepted_answer_count'] += 1

def is_new_user(user_id):
    return user_id not in users_df.id.values

In [6]:
data_path = '/media/antonio/92088d7f-1ed4-49dd-b55f-01462ab87ebb/so_data'

##Questions

In [7]:
xml_file = 'Questions-2014-01-0.xml'
xml_path = os.path.join(data_path, xml_file)

In [8]:
accepted_answer_ids = np.array([])

iterparser = etree.iterparse(xml_path, events=('start',))
next(iterparser)

elem_count = 0
t0 = time()
# count = 0
for event, elem in iterparser:
#     if count == 100:
#         break
#     count +=1
    
    
    add_question(elem)
    
    accepted_answer_id = elem.get('AcceptedAnswerId')
    is_accepted = accepted_answer_id is not None
    if is_accepted:
        accepted_answer_ids = np.append(accepted_answer_ids, accepted_answer_id)
    
    author_id = elem.get('OwnerUserId')
    if author_id is None:
        author_id = elem.get('OwnerDisplayName')
    if is_new_user(author_id):
        add_user(author_id)
    update_user_question_counts(author_id, answered_question=is_accepted)
    
    elem.clear()
    
    elem_count += 1
    if elem_count % 5000 == 0:
        print('%d elements processed' % elem_count)
t1 = time()

print('-' * 80)
print('Elements processed: %d' % elem_count)
print('Processing time: %d seconds' % (t1 - t0))
print('-' * 80)

5000 elements processed
10000 elements processed
15000 elements processed
20000 elements processed
25000 elements processed
30000 elements processed
35000 elements processed
40000 elements processed
45000 elements processed
50000 elements processed
--------------------------------------------------------------------------------
Elements processed: 52006
Processing time: 2111 seconds
--------------------------------------------------------------------------------


In [9]:
print('elem_count == len(questions_df) = %s' % (elem_count == len(questions_df)))

elem_count == len(questions_df) = True


In [10]:
questions_df.head()

,id,author_id,date,title,text,score,view_count,answer_count,comment_count,code_line_count,url
0,20864411,3150390,2014-01-01T00:00:20.607,nvd3.js How to make a historical multiBarChart?,"I have multiple, labeled sets of data with dat...",3,177,1,3,0,http://stackoverflow.com/questions/20864411
1,20864423,1377324,2014-01-01T00:02:46.990,Sphinx complex queries with mix of AND/OR,I've been looked around and have no luck on th...,0,44,1,0,7,http://stackoverflow.com/questions/20864423
2,20864424,2212490,2014-01-01T00:02:57.477,Google App Scripts get IP,How can I restrict a script on GAS to load onl...,1,127,1,0,0,http://stackoverflow.com/questions/20864424
3,20864427,2779244,2014-01-01T00:03:24.030,PHP form processing issues,I have been trying numerous ways to get the em...,-2,108,1,10,206,http://stackoverflow.com/questions/20864427
4,20864429,382775,2014-01-01T00:03:39.953,IAM policy to allow EC2 instance API access on...,I'm trying to set up an app that configures my...,0,686,1,0,16,http://stackoverflow.com/questions/20864429


In [11]:
questions_df.to_csv('data/questions.csv')

##Answers

In [12]:
xml_file = 'Answers-2014-01.xml'
xml_path = os.path.join(data_path, xml_file)

In [13]:
question_ids = np.array(questions_df['id'])

iterparser = etree.iterparse(xml_path, events=('start',))
next(iterparser)

elem_count = 0
t0 = time()
# count = 0
for event, elem in iterparser:
#     count +=1
#     if count == 1000:
#         break
    
    
    
    if elem.get('ParentId') not in question_ids:
        elem.clear()
        continue
    
    is_accepted = elem.get('Id') in accepted_answer_ids
    add_answer(elem, is_accepted)
    
    author_id = elem.get('OwnerUserId')
    if author_id is None:
        author_id = elem.get('OwnerDisplayName')
    if is_new_user(author_id):
        add_user(author_id)
    update_user_answer_counts(author_id, accepted_answer=is_accepted)
    
    elem.clear()
    
    elem_count += 1
    if elem_count % 5000 == 0:
        print('%d elements processed' % elem_count)
t1 = time()

print('-' * 80)
print('Elements processed: %d' % elem_count)
print('Processing time: %d seconds' % (t1 - t0))
print('-' * 80)

5000 elements processed
10000 elements processed
15000 elements processed
20000 elements processed
25000 elements processed
30000 elements processed
35000 elements processed
40000 elements processed
45000 elements processed
50000 elements processed
55000 elements processed
60000 elements processed
65000 elements processed
70000 elements processed
--------------------------------------------------------------------------------
Elements processed: 73746
Processing time: 4857 seconds
--------------------------------------------------------------------------------


In [14]:
print('elem_count == len(answers_df) = %s' % (elem_count == len(answers_df)))

elem_count == len(answers_df) = True


In [15]:
answers_df.head()

,id,author_id,question_id,date,text,score,accepted,comment_count,code_line_count,url
0,20864440,1455016,20864431,2014-01-01T00:07:10.900,This is a 1-character typo. The while loop wil...,2,False,4,9,http://stackoverflow.com/questions/20864431/20...
1,20864458,1491895,20864430,2014-01-01T00:11:32.343,Try this RE:\n\n\n\nChanges:\n\n\nThere's only...,0,True,0,8,http://stackoverflow.com/questions/20864430/20...
2,20864483,307266,20864457,2014-01-01T00:16:37.330,You can store the last known page index in a f...,1,True,0,8,http://stackoverflow.com/questions/20864457/20...
3,20864515,44729,20864454,2014-01-01T00:23:16.613,You can compile GL4 programs all day long. To...,6,True,3,0,http://stackoverflow.com/questions/20864454/20...
4,20864516,2960823,20864495,2014-01-01T00:23:40.403,Images also have an event. You can hook it to...,3,False,0,2,http://stackoverflow.com/questions/20864495/20...


In [16]:
answers_df.to_csv('data/answers.csv')

##Users

In [25]:
xml_file = 'Users-2014_2015-no_rep_1.xml'
xml_path = os.path.join(data_path, xml_file)

In [26]:
user_ids = np.array(users_df['id'])

iterparser = etree.iterparse(xml_path, events=('start',))
next(iterparser)

row_count = 0
elem_count = 0
t0 = time()
for event, elem in iterparser:
    
    row_count += 1
    if row_count % 100000 == 0:
        print('%d rows' % row_count)
    
    
    if elem.get('Id') not in user_ids:
        elem.clear()
        continue
    
    update_user_info(elem)
    
    elem.clear()
    
    elem_count += 1
    if elem_count % 5000 == 0:
        print('%d elements processed' % elem_count)
t1 = time()

print('-' * 80)
print('Elements processed: %d' % elem_count)
print('Processing time: %d seconds' % (t1 - t0))
print('-' * 80)

5000 elements processed
100000 rows
10000 elements processed
200000 rows
15000 elements processed
300000 rows
20000 elements processed
400000 rows
25000 elements processed
500000 rows
30000 elements processed
600000 rows
35000 elements processed
40000 elements processed
700000 rows
45000 elements processed
800000 rows
900000 rows
1000000 rows
--------------------------------------------------------------------------------
Elements processed: 49520
Processing time: 4627 seconds
--------------------------------------------------------------------------------


In [27]:
print('elem_count == len(users_df) = %s' % (elem_count == len(users_df)))

elem_count == len(users_df) = False


In [28]:
elem_count

49520

In [29]:
len(users_df)

53283

In [30]:
users_df.head()

,id,date,name,reputation,total_question_count,answered_question_count,total_answer_count,accepted_answer_count,url
0,3150390,2013-12-31T23:45:57.020,user3150390,16,1,0,0,0,http://stackoverflow.com/users/3150390
1,1377324,2012-05-05T20:45:40.307,user1377324,42,1,0,0,0,http://stackoverflow.com/users/1377324
2,2212490,2013-03-26T16:38:08.430,VEC,28,2,0,0,0,http://stackoverflow.com/users/2212490
3,2779244,2013-09-14T13:19:37.960,user252415,9,2,1,0,0,http://stackoverflow.com/users/2779244
4,382775,2010-07-03T17:01:27.020,dmikalova,56,1,0,0,0,http://stackoverflow.com/users/382775


In [31]:
users_df.to_csv('data/users.csv')

##Question-Answer

In [53]:
qs_df = questions_df[['id', 'author_id', 'title', 'text', 'score', 'view_count', 'answer_count', 'comment_count', 'code_line_count']]
as_df = answers_df[['id', 'author_id', 'question_id', 'text', 'comment_count', 'code_line_count', 'score', 'accepted']]
us_df = users_df[['id', 'reputation', 'total_question_count', 'answered_question_count', 'total_answer_count', 'accepted_answer_count']]

In [54]:
qus_df = pd.merge(qs_df, us_df, left_on='author_id', right_on='id', suffixes=('_q', '_u')).drop('id_u', axis=1)
aus_df = pd.merge(as_df, us_df, left_on='author_id', right_on='id', suffixes=('_a', '_u')).drop('id_u', axis=1)
qas_df = pd.merge(qus_df, aus_df, left_on='id_q', right_on='question_id', suffixes=('_q', '_a')).drop('question_id', axis=1)

In [55]:
# id = id_q-id_a
qas_df['id'] = qas_df[['id_q', 'id_a']].apply(lambda x: x['id_q'] + '-' + x['id_a'], axis=1)

# text = title - text_q -- text_a
qas_df['text'] = qas_df[['title', 'text_q', 'text_a']].apply(lambda x: x['title'] + ' - ' + x['text_q'] + ' -- ' + x['text_a'], axis=1)

# percent_answered_questions_q = answered_question_count_q * 100 / total_question_count_q
qas_df['percent_answered_questions_q'] = qas_df[['answered_question_count_q', 'total_question_count_q']].apply(lambda x: round(100 * x['answered_question_count_q'] / x['total_question_count_q']), axis=1)

# percent_accepted_answers_a = accepted_answer_count_a * 100 / total_answer_count_a
qas_df['percent_accepted_answers_a'] = qas_df[['accepted_answer_count_a', 'total_answer_count_a']].apply(lambda x: round(100 * x['accepted_answer_count_a'] / x['total_answer_count_a']), axis=1)

cols_to_delete = ['author_id_q', 'author_id_a', 'total_answer_count_q', 'accepted_answer_count_q', 'total_question_count_a', 'answered_question_count_a']
cols_to_delete.extend(['id_q', 'id_a'])
cols_to_delete.extend(['title', 'text_q', 'text_a'])
cols_to_delete.extend(['answered_question_count_q', 'total_question_count_q'])
cols_to_delete.extend(['accepted_answer_count_a', 'total_answer_count_a'])

qas_df.drop(cols_to_delete, axis=1, inplace=True)

print('-' * 80)
print('Total q-a pairs: %d' % elem_count)
print('-' * 80)

--------------------------------------------------------------------------------
Total q-a pairs: 49520
--------------------------------------------------------------------------------


In [56]:
qas_df.head()

,score_q,view_count,answer_count,comment_count_q,code_line_count_q,reputation_q,comment_count_a,code_line_count_a,score_a,accepted,reputation_a,id,text,percent_answered_questions_q,percent_accepted_answers_a
0,3,177,1,3,0,16,4,3,0,False,947,20864411-20864961,nvd3.js How to make a historical multiBarChart...,0,15
1,0,44,1,0,7,42,0,3,0,False,11986,20864423-20864519,Sphinx complex queries with mix of AND/OR - I'...,0,50
2,1,127,1,0,0,28,0,0,0,False,6920,20864424-20864588,Google App Scripts get IP - How can I restrict...,0,33
3,2,68,2,0,2,28,0,0,1,False,1055,20982426-20995560,How to use a different email with MailApp in G...,0,40
4,2,68,2,0,2,28,1,0,0,False,21855,20982426-21002137,How to use a different email with MailApp in G...,0,33


In [57]:
users_df.to_csv('data/qas.csv')